In [1]:
def preprocess(text):
  text = text.lower()
  text = text.replace('.', ' .')
  words = text.split(' ')

  word_id_id = {}
  id_to_word = {}

  for word in words:
    if word not in word_to_id:
      new_id = len(word_to_id)
      word_to_id[word] = new_id
      id_to_word[new_id] = word_to_id

  corpus = np.array([word_to_id[w] for w in words])

  return corpus, word_to_id, id_to_word

In [2]:
def create_co_matrix(corpus, vocab_size, window_size=1):
  corpus_size = len(corpus)
  co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

  for idx, word_id in enumerate(corpus):
      for i in range(1, window_size + 1):
          left_idx = idx - i
          right_idx = idx + i

          if left_idx >= 0:
              left_word_id = corpus[left_idx]
              co_matrix[word_id, left_word_id] += 1

          if right_idx < corpus_size:
              right_word_id = corpus[right_idx]
              co_matrix[word_id, right_word_id] += 1

  return co_matrix

In [3]:
def cos_similarity(x, y, eps=1e-8):
  nx = x / (np.sqrt(np.sum(x ** 2)) + eps)
  ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
  return np.dot(nx, ny)

In [4]:
def ppmi(C, verbose=False, eps=1e-8):
  M = np.zeros_like(C, dtype=np.float32)
  N = np.sum(C)
  S = np.sum(C, axis=0)
  total = C.shape[0] * C.shape[1]
  cnt = 0

  for i in range(C.shape[0]):
      for j in range(C.shape[1]):
          pmi = np.log2(C[i, j] * N / (S[j]*S[i]) + eps)
          M[i, j] = max(0, pmi)

          if verbose:
              cnt += 1
              if cnt % (total//100 + 1) == 0:
                  print('%.1f%% done' % (100*cnt/total))
  return M

In [6]:
import sys
sys.path.append('..')
import numpy as np
#from common.util import preprocess, create_co_matrix, cos_similarity, ppmi

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
W = ppmi(C)

np.set_printoptions(precision=3)
print('covariance matrix')
print(C)
print('-'*50)
print('PPMI')
print(W)

NameError: ignored